In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import pandas as pd
import numpy as np
import math
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
import tensorflow as tf
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import datetime
from sklearn.model_selection import train_test_split

In [ ]:
embed_size = 50 
max_features = 20000 
maxlen = 250 

In [ ]:
df_final_temp=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/mini_datasets/final_dataset.csv")

categorywise_data = df_final_temp[['genuine','social_spambot_1','social_spambot_2', 'social_spambot_3','traditional_spambot','fake_followers']] 
counts_category = []                                              
categories = list(categorywise_data.columns.values)
for i in categories:
    counts_category.append((i, categorywise_data[i].sum()))
    
dataframe = pd.DataFrame(counts_category, columns=['Labels', 'number_of_comments'])

EMBEDDING_FILE = '/content/drive/MyDrive/SAAS-2121/Bot_Ranking_Code/Hate_Speech_Detection/glove.6B.50d.txt'
del df_final_temp["Unnamed: 0"]

df_final_temp=df_final_temp.drop_duplicates(subset='text')

In [ ]:
def remove_urls(string):
    urls=re.compile(r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    for url in re.findall(urls, string):
      string=re.sub(url, "<URL>", string)
    
    return string

def remove_tags(string):
    tags=re.compile(r'(^|[^@\w])@(\w{1,15})\b')
    for tag in re.findall(tags, string):
      account_name='@'+tag[1]
      string=re.sub(account_name, "<ACCOUNT>", string)
  
    return string

def clean_tweets(tweets):
  tweets_without_urls_and_tags = []
  for tweet in list(tweets):
    try:
      tweet_without_urls_and_tags = remove_tags(remove_urls(tweet))
      tweets_without_urls_and_tags.append(tweet_without_urls_and_tags)
    except:
      tweet=''
      tweets_without_urls_and_tags.append(tweet)

  return tweets_without_urls_and_tags


In [ ]:
## cleaning the tweets, renaming and reordering columns 
df_final_temp['cleaned_tweets'] = clean_tweets(df_final_temp['text'])
df_final_temp.drop('text', axis=1, inplace=True)
df_final_temp.rename(columns={"cleaned_tweets":"text"}, inplace=True)
cols = df_final_temp.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_final_temp = df_final_temp[cols]
df_final_temp

,text,genuine,social_spambot_1,social_spambot_2,social_spambot_3,traditional_spambot,fake_followers
0,RT <ACCOUNT>: #KaptaanKhanOnGeo,1,0,0,0,0,0
5,Brain fitness seen as hot industry of the futu...,0,0,0,0,1,0
6,"""'Twas the Night After Edmund"" available on IT...",0,0,0,0,0,1
8,Chuva chata tudo praga do <ACCOUNT>,0,0,0,0,0,1
16,Flood-hit town gets safety support - <URL>,0,0,0,0,1,0
...,...,...,...,...,...,...,...
210408,I don't think going through all his fetus musi...,0,0,1,0,0,0
210415,My lrt&amp;quote thats just like me!,0,0,1,0,0,0
210423,kiwi doesn't know what a penis is,0,0,1,0,0,0
210434,don't keep askin for my advice if you don't li...,0,0,1,0,0,0


In [ ]:
train, test = train_test_split(df_final_temp, test_size=0.2)

In [ ]:
train_x=train["text"]
test_x=test["text"]
train_x = train['text'].str.lower()
test_x = test['text'].str.lower()

In [ ]:
train_y = train[['genuine','social_spambot_1','social_spambot_2', 'social_spambot_3','traditional_spambot','fake_followers']].values
test_y=test[['genuine','social_spambot_1','social_spambot_2', 'social_spambot_3','traditional_spambot','fake_followers']].values
df_final_temp=df_final_temp.applymap(str)

In [ ]:
list_sentences_train=train_x
list_sentences_test =test_x

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.020940498, 0.6441043)

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_t, train_y, batch_size=32, epochs=2, validation_split=0.1);

Epoch 1/2
876/876 [==============================] - 512s 578ms/step - loss: 0.2694 - accuracy: 0.6207 - val_loss: 0.2114 - val_accuracy: 0.7052
Epoch 2/2
876/876 [==============================] - 513s 585ms/step - loss: 0.1919 - accuracy: 0.7401 - val_loss: 0.1809 - val_accuracy: 0.7624


In [ ]:
df_geuine=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/genuine_accounts.csv/tweets.csv", error_bad_lines=False)
df_social_1=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_1.csv/tweets.csv", error_bad_lines=False)
df_social_2=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_2.csv/tweets.csv", error_bad_lines=False)
df_social_3=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/social_spambots_3.csv/tweets.csv", error_bad_lines=False)
df_traditional=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/traditional_spambots_1.csv/tweets.csv", error_bad_lines=False)
df_fake_followers=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/datasets_full.csv/fake_followers.csv/tweets.csv", error_bad_lines=False)


NameError: ignored

In [ ]:
df_genuine_part=pd.DataFrame()
df_genuine_part["text"]=df_geuine["RT @morningJewshow: Speaking about Jews and comedy tonight at Temple Emanu-El in San Francisco. In other words, my High Holidays."]
df_genuine_part["genuine"]=1
df_genuine_part["social_spambot_1"]=0
df_genuine_part["social_spambot_2"]=0
df_genuine_part["social_spambot_3"]=0
df_genuine_part["traditional_spambot"]=0
df_genuine_part["fake_followers"]=0

In [ ]:
df_social_1_part=pd.DataFrame()
df_social_1_part["text"]=df_social_1["text"]
df_social_1_part["genuine"]=0
df_social_1_part["social_spambot_1"]=1
df_social_1_part["social_spambot_2"]=0
df_social_1_part["social_spambot_3"]=0
df_social_1_part["traditional_spambot"]=0
df_social_1_part["fake_followers"]=0

In [ ]:
df_social_1_part=pd.DataFrame()
df_social_2_part["text"]=df_social_2["text"]
df_social_2_part["genuine"]=0
df_social_2_part["social_spambot_1"]=0
df_social_2_part["social_spambot_2"]=1
df_social_2_part["social_spambot_3"]=0
df_social_2_part["traditional_spambot"]=0
df_social_2_part["fake_followers"]=0

In [ ]:
df_social_3_part=pd.DataFrame()
df_social_3_part["text"]=df_social_3["text"]
df_social_3_part["genuine"]=0
df_social_3_part["social_spambot_1"]=0
df_social_3_part["social_spambot_2"]=0
df_social_3_part["social_spambot_3"]=1
df_social_3_part["traditional_spambot"]=0
df_social_3_part["fake_followers"]=0

In [ ]:
df_traditional_part=pd.DataFrame()
df_traditional_part["text"]=df_traditional["text"]
df_traditional_part["genuine"]=0
df_traditional_part["social_spambot_1"]=0
df_traditional_part["social_spambot_2"]=0
df_traditional_part["social_spambot_3"]=0
df_traditional_part["traditional_spambot"]=1
df_traditional_part["fake_followers"]=0

In [ ]:
df_fake_followers_part=pd.DataFrame()
df_fake_followers_part["text"]=df_fake_followers["text"]
df_fake_followers_part["genuine"]=0
df_fake_followers_part["social_spambot_1"]=0
df_fake_followers_part["social_spambot_2"]=0
df_fake_followers_part["social_spambot_3"]=0
df_fake_followers_part["traditional_spambot"]=0
df_fake_followers_part["fake_followers"]=1

In [ ]:
"""df_genuine_part.to_csv("genuine_full.csv")
df_social_1_part.to_csv("socialspam_1_full.csv")
df_social_2_part.to_csv("socialspam_2_full.csv")
df_social_3_part.to_csv("socialspam_3_full.csv")
df_traditional_part.to_csv("traditional_full.csv")
df_fake_followers_part.to_csv("fakefollowers_full.csv")"""
df_genuine_part=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/tweet_level dataset/genuine_full.csv",engine='python')
df_social_1_part=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/tweet_level dataset/socialspam_1_full.csv",engine='python')
df_social_2_part=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/tweet_level dataset/socialspam_2_full.csv",engine='python')
df_social_3_part=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/tweet_level dataset/socialspam_3_full.csv",engine='python')
df_traditional_part=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/tweet_level dataset/traditional_full.csv",engine='python')
df_fake_followers_part=pd.read_csv("/content/drive/MyDrive/SAAS-2121/Datasets/tweet_level dataset/fakefollowers_full.csv",engine='python')

In [ ]:
final_tweet_dataset=pd.DataFrame()

In [ ]:
final_tweet_dataset=final_tweet_dataset.append(df_genuine_part)
final_tweet_dataset=final_tweet_dataset.append(df_social_1_part)
final_tweet_dataset=final_tweet_dataset.append(df_social_2_part)
final_tweet_dataset=final_tweet_dataset.append(df_social_3_part)
final_tweet_dataset=final_tweet_dataset.append(df_traditional_part)
final_tweet_dataset=final_tweet_dataset.append(df_fake_followers_part)